<a href="https://colab.research.google.com/github/sudhang/arangodb-cugraph-hackathon/blob/master/MakeNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Construct Network

In [1]:
%pip install pandas tqdm openai

In [2]:
import pandas as pd
import openai
import json
import networkx as nx
from networkx.readwrite import json_graph

from tqdm import tqdm
from google.colab import userdata


openai.api_key = userdata.get('OPENAI_API_KEY')

Load the raw dataframes.  There is a dataframe of the top physics papers that have been written and a dataframe of authors who wrote them.

In [3]:

from google.colab import userdata, drive

drive.mount('/content/drive')

DF_papers = pd.read_csv('/content/drive/MyDrive/TU/arangodbhackathon/data/physics_papers.csv.tar.bz2', delimiter="|", compression='bz2')
DF_papers.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Filtered_DF_top_1000_papers_Physics_Filtered_DF_top_1000_papers_Physics_cleaned.csv,id,year,fieldsOfStudy,authors,doi,paperAbstract,outCitations,inCitations,journalName,N_citations
0,0.0,6316c3eac036defb65a172bd9a7b150a56b69895,1996.0,"['Physics', 'Medicine']","[{'name': 'Perdew', 'ids': ['121583389']}, {'n...",10.1103/PHYSREVLETT.77.3865,Generalized gradient approximations (GGA’s) fo...,"['776f6d45b38cf048ff30cea41f0cd3ceeefed622', '...","['90f2fc9dde7ad46ee8fb84a81811185cf48a896a', '...",Physical review letters,89761.0
1,1.0,09a356721104fab82fafd86fc0d3296b70f635fe,1988.0,"['Physics', 'Medicine']","[{'name': 'Lee', 'ids': ['1409527728']}, {'nam...",10.1103/PHYSREVB.37.785,A correlation-energy formula due to Colle and ...,[],"['657c29715eaa8656cb9f02ccdac3ef80fe3886d3', '...","Physical review. B, Condensed matter",61498.0
2,2.0,ed73eb5b0fcb517d1e13feaa1e09be163e7f7cde,1996.0,"['Physics', 'Medicine']","[{'name': 'Kresse', 'ids': ['30170920']}, {'na...",10.1103/PHYSREVB.54.11169,We present an efficient scheme for calculating...,['de8a94643d3322d7136e0cbbe84a549b4a9f880c'],"['b47cb46a992315936227fdcc82b7b34deb005edc', '...","Physical review. B, Condensed matter",48421.0
3,3.0,c92bd747a97eeafdb164985b0d044caa1dc6e73e,2004.0,"['Chemistry', 'Physics', 'Medicine']","[{'name': 'K S Novoselov', 'ids': ['144100574'...",10.1126/SCIENCE.1102896,"We describe monocrystalline graphitic films, w...","['ebef523c81a685e695202c99c731e9ee47c6a341', '...","['a64a3ef345e4124e8f22f9067cde4264493ea57b', '...",Science,40599.0
4,4.0,daef35e67a35db44702cdd894e11c2fdc98de83d,1994.0,"['Physics', 'Medicine']","[{'name': 'Blöchl', 'ids': ['116676123']}]",10.1103/PHYSREVB.50.17953,An approach for electronic structure calculati...,[],"['b36c48122193127c0b9c590b0c49b1785d39ec35', '...","Physical review. B, Condensed matter",36132.0


In [4]:
from google.colab import userdata
import openai
import time

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=OPENAI_API_KEY)

def generate_title(abstract):
    """Generate a short title using OpenAI API."""
    if not isinstance(abstract, str) or len(abstract.strip()) == 0:
        return "Unknown Title"

    prompt = f"Generate a concise, informative title (5-7 words) for this research abstract:\n\n{abstract}\n\nTitle:"

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a scientific title generator."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=15,
            temperature=0.7
        )
        the_title = response.choices[0].message.content.strip()
        print(f"{the_title=}")
        return the_title
    except Exception as e:
        print(f"Error generating title: {e}")
        return "Title Generation Failed"

# Process abstracts in batches of 100
def generate_titles_for_all(df, batch_size=100):
    """Generate titles for all abstracts in batches."""
    titles = []
    for i in tqdm(range(0, len(df), batch_size), desc="Generating Titles"):
        batch = df.iloc[i:i+batch_size]
        batch_titles = [generate_title(abstract) for abstract in batch['paperAbstract']]
        titles.extend(batch_titles)
        time.sleep(2)  # To avoid rate limits
    return titles

# Generate titles
DF_papers['generatedTitle'] = generate_titles_for_all(DF_papers)


Generating Titles:   0%|          | 0/12 [00:00<?, ?it/s]

the_title='"Fundamental Constants-Based Generalized Gradient Approximations for DFT"'
the_title='Density-Functional Formulas for Correlation Energy Calculation'
the_title='Efficient Kohn-Sham Ground State Calculation for Metallic Systems'
the_title='"Monocrystalline Graphitic Films: 2D Semim'
the_title='"Generalized Electronic Structure Method for Molecular Dynamics"'
the_title='Interplay of Ultrasoft Pseudopotentials and PAW Method'
the_title='Approximation Methods for Inhomogeneous Electron Systems'
the_title='"The Enduring Influence of The Structure of Scientific Revolutions"'
the_title='"Efficient Point Generation in Brillouin Zone Integration"'
the_title='"Efficient Monte Carlo Method for Molecular Equations"'
the_title='"Graphene: Unveiling New Physics and Applications"'
the_title='"Emergence of Scale-Free Networks: Self-Organizing Phenomena'
the_title='"Universal Density Functional for Interacting Electron Gas"'
the_title='"Integrating Data Types in Bayesian Phylogenetics"'
the_

Generating Titles:   8%|▊         | 1/12 [00:35<06:34, 35.84s/it]

the_title='"Exploring Liquid Properties: Statistical Mechanics to Transport"'
the_title='Radiation Detection and Spectroscopy Techniques Overview'
the_title='"Solar Radiation Fundamentals and Applications: A Review"'
the_title='"Advances in Liquid Crystal Phase Behavior"'
the_title='"Modern Pedagogic Introduction to Quantum Field Theory"'
the_title='"Parameterized Extinction Data for Interstellar Medium"'
the_title='"Experimental Confirmation of Negative Refraction in Metamaterials"'
the_title='"Unified Molecular Dynamics and Density-Functional Theory"'
the_title='"Functional Connectivity of Resting Human Brain: MRI Insights"'
the_title='"Crystal Lattice Dynamics: Definitive Insights and Analysis"'
the_title='"Quantum Gravity Effects: Hawking, Early Universe, Calculations"'
the_title='"Herbert Spencer\'s Influence on Evolutionary Philosophy"'
the_title='"Evolutionary Applications of Game Theory in Ecology"'
the_title='"Statistical Framework for Detecting Power-law Distributions"'
the_t

Generating Titles:  17%|█▋        | 2/12 [01:17<06:31, 39.20s/it]

...

Generating Titles:  75%|███████▌  | 9/12 [05:29<01:49, 36.65s/it]

the_title='"Quantum Mechanics: Theory and Practical Applications"'
the_title='"Quantum Coherence Enhances Energy Transfer Efficiency"'
the_title='"Unified Confidence Belt Construction for Classical Statistics"'
the_title='"Accurate Pseudopotential Extraction for Atomic Calculations"'
the_title='"Improved Astronomical Computation of Earth\'s Insolation"'
the_title='"Universality in Anomalous Carrier Transport Dynamics"'
the_title='"Modeling Electron Motion in Lattice Structures"'
the_title='"Dynamic Valley Polarization in Monolayer MoS2"'
the_title='"Novel NdFeB Compound with Superior Magnetic Properties"'
the_title='"Comparison of GW and TDDFT Methods for Electronic Excitations"'
the_title='"Digitalización del material de la Fundación Antorchas"'
the_title='"Astronomical Spectra of Cosmic Radiation Sources"'
the_title='"Conservation of P and T in Color Gauge Theory"'
the_title='"Vortex Scaling and Energy Transfer in Turbulence"'
the_title='"Doping of Mott Insulators in Cuprates: A Comp

Generating Titles:  83%|████████▎ | 10/12 [06:05<01:12, 36.22s/it]

the_title='"High-Intensity Source for Polarization-Entangled Photon Pairs"'
the_title='"Generalized Theory for Elastic-Plastic Deformation Kinematics"'
the_title='"Quantum Phase Transition in Bosonic Atoms"'
the_title='"Unified Long-Wave Theory for Thin Liquid Films"'
the_title='"Multi-Dimensional Entanglement: Expanding Quantum Communication Channels"'
the_title='"Enhancing Micromachine Resolution via Two-Photon Absorption"'
the_title='"Evolution of Electron Transfer Processes: History and Trends"'
the_title='"Enhancing Forecast Models with Nonlocal Diffusion"'
the_title='"Reconsidering Hidden Mass in Galaxies: Modified Dynamics"'
the_title='"Optical Fiber Coupled Mode Theory Applications"'
the_title='"Enhancements in ISCCP Cloud Data Analysis"'
the_title='"Prospects for Detecting Neutralino Dark Matter"'
the_title='"Coherent Forward Scattering in Neutrino Oscillations"'
the_title='"Nonlinear Gravitational Clustering in Cold Dark Matter Universes:'
the_title='"Relativistic Dual-Space 

Generating Titles: 100%|██████████| 12/12 [06:44<00:00, 33.68s/it]


In [5]:
DF_papers.head()

,Filtered_DF_top_1000_papers_Physics_Filtered_DF_top_1000_papers_Physics_cleaned.csv,id,year,fieldsOfStudy,authors,doi,paperAbstract,outCitations,inCitations,journalName,N_citations,generatedTitle
0,0.0,6316c3eac036defb65a172bd9a7b150a56b69895,1996.0,"['Physics', 'Medicine']","[{'name': 'Perdew', 'ids': ['121583389']}, {'n...",10.1103/PHYSREVLETT.77.3865,Generalized gradient approximations (GGA’s) fo...,"['776f6d45b38cf048ff30cea41f0cd3ceeefed622', '...","['90f2fc9dde7ad46ee8fb84a81811185cf48a896a', '...",Physical review letters,89761.0,"""Fundamental Constants-Based Generalized Gradi..."
1,1.0,09a356721104fab82fafd86fc0d3296b70f635fe,1988.0,"['Physics', 'Medicine']","[{'name': 'Lee', 'ids': ['1409527728']}, {'nam...",10.1103/PHYSREVB.37.785,A correlation-energy formula due to Colle and ...,[],"['657c29715eaa8656cb9f02ccdac3ef80fe3886d3', '...","Physical review. B, Condensed matter",61498.0,Density-Functional Formulas for Correlation En...
2,2.0,ed73eb5b0fcb517d1e13feaa1e09be163e7f7cde,1996.0,"['Physics', 'Medicine']","[{'name': 'Kresse', 'ids': ['30170920']}, {'na...",10.1103/PHYSREVB.54.11169,We present an efficient scheme for calculating...,['de8a94643d3322d7136e0cbbe84a549b4a9f880c'],"['b47cb46a992315936227fdcc82b7b34deb005edc', '...","Physical review. B, Condensed matter",48421.0,Efficient Kohn-Sham Ground State Calculation f...
3,3.0,c92bd747a97eeafdb164985b0d044caa1dc6e73e,2004.0,"['Chemistry', 'Physics', 'Medicine']","[{'name': 'K S Novoselov', 'ids': ['144100574'...",10.1126/SCIENCE.1102896,"We describe monocrystalline graphitic films, w...","['ebef523c81a685e695202c99c731e9ee47c6a341', '...","['a64a3ef345e4124e8f22f9067cde4264493ea57b', '...",Science,40599.0,"""Monocrystalline Graphitic Films: 2D Semim"
4,4.0,daef35e67a35db44702cdd894e11c2fdc98de83d,1994.0,"['Physics', 'Medicine']","[{'name': 'Blöchl', 'ids': ['116676123']}]",10.1103/PHYSREVB.50.17953,An approach for electronic structure calculati...,[],"['b36c48122193127c0b9c590b0c49b1785d39ec35', '...","Physical review. B, Condensed matter",36132.0,"""Generalized Electronic Structure Method for M..."


In [6]:
# Where did the extra quotes around the generatedTitle appear from?  Remove them
DF_papers['generatedTitle'] = DF_papers['generatedTitle'].str.replace('"', '')
DF_papers.head()

,Filtered_DF_top_1000_papers_Physics_Filtered_DF_top_1000_papers_Physics_cleaned.csv,id,year,fieldsOfStudy,authors,doi,paperAbstract,outCitations,inCitations,journalName,N_citations,generatedTitle
0,0.0,6316c3eac036defb65a172bd9a7b150a56b69895,1996.0,"['Physics', 'Medicine']","[{'name': 'Perdew', 'ids': ['121583389']}, {'n...",10.1103/PHYSREVLETT.77.3865,Generalized gradient approximations (GGA’s) fo...,"['776f6d45b38cf048ff30cea41f0cd3ceeefed622', '...","['90f2fc9dde7ad46ee8fb84a81811185cf48a896a', '...",Physical review letters,89761.0,Fundamental Constants-Based Generalized Gradie...
1,1.0,09a356721104fab82fafd86fc0d3296b70f635fe,1988.0,"['Physics', 'Medicine']","[{'name': 'Lee', 'ids': ['1409527728']}, {'nam...",10.1103/PHYSREVB.37.785,A correlation-energy formula due to Colle and ...,[],"['657c29715eaa8656cb9f02ccdac3ef80fe3886d3', '...","Physical review. B, Condensed matter",61498.0,Density-Functional Formulas for Correlation En...
2,2.0,ed73eb5b0fcb517d1e13feaa1e09be163e7f7cde,1996.0,"['Physics', 'Medicine']","[{'name': 'Kresse', 'ids': ['30170920']}, {'na...",10.1103/PHYSREVB.54.11169,We present an efficient scheme for calculating...,['de8a94643d3322d7136e0cbbe84a549b4a9f880c'],"['b47cb46a992315936227fdcc82b7b34deb005edc', '...","Physical review. B, Condensed matter",48421.0,Efficient Kohn-Sham Ground State Calculation f...
3,3.0,c92bd747a97eeafdb164985b0d044caa1dc6e73e,2004.0,"['Chemistry', 'Physics', 'Medicine']","[{'name': 'K S Novoselov', 'ids': ['144100574'...",10.1126/SCIENCE.1102896,"We describe monocrystalline graphitic films, w...","['ebef523c81a685e695202c99c731e9ee47c6a341', '...","['a64a3ef345e4124e8f22f9067cde4264493ea57b', '...",Science,40599.0,Monocrystalline Graphitic Films: 2D Semim
4,4.0,daef35e67a35db44702cdd894e11c2fdc98de83d,1994.0,"['Physics', 'Medicine']","[{'name': 'Blöchl', 'ids': ['116676123']}]",10.1103/PHYSREVB.50.17953,An approach for electronic structure calculati...,[],"['b36c48122193127c0b9c590b0c49b1785d39ec35', '...","Physical review. B, Condensed matter",36132.0,Generalized Electronic Structure Method for Mo...


Now, let's try to look at the affiliations of the authors

In [7]:
DF_affiliations = pd.read_csv('/content/drive/MyDrive/TU/arangodbhackathon/data/affiliations.csv.tar.bz2', delimiter="|", compression='bz2')

# Drop missing DOIs
DF_affiliations.dropna(subset=["doi"], inplace=True)

# Add year to affiliations (matching by DOI)
DF_affiliations = DF_affiliations.merge(DF_papers[['doi', 'year']], on='doi', how='left')

DF_affiliations.head()

,affiliations.csv,aff_city,aff_city_id,aff_country,aff_country_code,aff_id,aff_name,aff_raw_affiliation,aff_state,aff_state_code,pub_id,researcher_id,first_name,last_name,doi,Nationality,Gender,year
0,0.0,New Orleans,4335045.0,United States,US,grid.265219.b,Tulane University,Department of Physics and Quantum Theory Group...,Louisiana,US-LA,pub.1060814179,ur.01153754354.91,John P,Perdew,10.1103/PHYSREVLETT.77.3865,US,male,1996.0
1,1.0,New Orleans,4335045.0,United States,US,grid.265219.b,Tulane University,Department of Physics and Quantum Theory Group...,Louisiana,US-LA,pub.1060814179,ur.01270235327.24,Kieron,Burke,10.1103/PHYSREVLETT.77.3865,IE,male,1996.0
2,2.0,New Orleans,4335045.0,United States,US,grid.265219.b,Tulane University,Department of Physics and Quantum Theory Group...,Louisiana,US-LA,pub.1060814179,ur.01326610423.52,Matthias,Ernzerhof,10.1103/PHYSREVLETT.77.3865,DE,male,1996.0
3,3.0,Chapel Hill,4460162.0,United States,US,grid.410711.2,University of North Carolina System,Department of Chemistry| University of North C...,North Carolina,US-NC,pub.1060546104,ur.07541605423.18,Chengteh,Lee,10.1103/PHYSREVB.37.785,KR,NaN,1988.0
4,4.0,Chapel Hill,4460162.0,United States,US,grid.410711.2,University of North Carolina System,Department of Chemistry| University of North C...,North Carolina,US-NC,pub.1060546104,ur.01201000573.87,Weitao,Yang,10.1103/PHYSREVB.37.785,CN,male,1988.0


## Make Bipartite Graph

In [13]:
G = nx.Graph()

for _, row in DF_papers.iterrows():
    G.add_node(row['doi'],
               type='Paper',
               title=row.get('generatedTitle', ''),
               year=row.get('year', None),
               num_citations=row.get('N_citations', 0),
               abstract=row.get('paperAbstract', ''))


for _, row in DF_affiliations.iterrows():
    G.add_node(row['researcher_id'],
               type='Author',
               name=f"{row.get('first_name', '')} {row.get('last_name', '')}".strip(),
               affiliation=row.get('aff_name', ''),
               country=row.get('aff_country', ''),
               gender=row.get('Gender', ''),
               num_papers=0,
               num_citations=0,
               degree=0)


for _, row in DF_affiliations.iterrows():
    G.add_edge(row['researcher_id'], row['doi'], relation='WRITTEN_BY')


co_authorship_counts = {}
for doi, authors in DF_affiliations.groupby("doi")["researcher_id"].apply(set).items():
    for author1 in authors:
        for author2 in authors:
            if author1 != author2:
                if (author1, author2) in co_authorship_counts:
                    co_authorship_counts[(author1, author2)] += 1
                else:
                    co_authorship_counts[(author1, author2)] = 1


for (author1, author2), weight in co_authorship_counts.items():
    G.add_edge(author1, author2, relation='CO_AUTHOR_WITH', weight=weight)


for _, row in DF_papers.iterrows():
    cited_papers = eval(row['outCitations']) if isinstance(row['outCitations'], str) else []
    for cited_paper in cited_papers:
        if cited_paper in G.nodes:
            G.add_edge(row['doi'], cited_paper, relation='CITES')


for author in [n for n, d in G.nodes(data=True) if d['type'] == 'Author']:
    written_papers = list(G.neighbors(author))
    citations = sum(G.nodes[paper]['num_citations'] for paper in written_papers if paper in G.nodes)
    productivity = len(written_papers)
    degree = len(list(G.neighbors(author)))  # Number of direct co-authors

    nx.set_node_attributes(G, {author: {'num_papers': productivity, 'num_citations': citations, 'degree': degree}})


### Export Graph

For consumption by Hackathon Notebook

In [12]:
nx.write_graphml(G, "coauthorship_bipartite.graphml")
nx.write_gml(G, "coauthorship_bipartite.gml")


# Convert NetworkX graph to node-link format
graph_data = json_graph.node_link_data(G)

# Save to JSON file
with open("coauthorship_bipartite.json", "w") as f:
    json.dump(graph_data, f)

/usr/local/lib/python3.11/dist-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(
